In [ ]:
import json
from pathlib import Path
from collections import defaultdict
import statistics

In [ ]:
datasets = ["temporal-nli", "matres", "tddiscourse", "torque"]
model_ids = ["Llama-2-7b-hf", "Llama-2-7b-chat-hf", "Llama-2-13b-hf", "Llama-2-70b-hf", "flan-t5-xl", "flan-t5-xxl", "t5-3b"]

In [ ]:
dirpath = Path("../output_score/benchmark/")
scores = defaultdict(lambda: defaultdict(list))
for dataset in datasets:
    print(dataset)
    for model_id in model_ids:
        print(f"  {model_id}")
        for filepath in dirpath.glob(f"{dataset}/*{model_id}*few-shot*/*.json"):
            
            with open(filepath, "r") as f:
                data = json.load(f)
            
            if dataset == "temporal-nli":
                _max = statistics.median(
                    [x['macro avg']['f1-score'] for x in data['individuals']['example-wise-scores'].values()]
                )
                scores[dataset][model_id].append(_max)
            elif dataset == 'matres':
                # for k, x in data['individuals']['example-wise-scores'].items():
                #     if 'micro avg' not in x:
                #         print(filepath)
                #         print(k)
                #         print(x)
                _max = statistics.median(
                    [x['micro avg']['f1-score'] if 'micro avg' in x else x['accuracy'] for x in data['individuals']['example-wise-scores'].values()]
                )
                scores[dataset][model_id].append(_max)
            else:
                if dataset == "torque":
                    _max = data['average']['exact-match-relaxed']['median']
                else:
                    _max = data['average']['example-wise-scores']['median']
                scores[dataset][model_id].append(_max)

In [ ]:
for dataset, _scores in scores.items():
    print(f"{dataset}")
    for model, __scores in _scores.items():
        print(f"  {model}: {max(__scores):.3f}")